In [ ]:
!pip3 install SpeechRecognition pydub

     |████████████████████████████████| 32.8 MB 1.2 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 8.7 MB/s 
     |████████████████████████████████| 3.3 MB 54.4 MB/s 
     |████████████████████████████████| 596 kB 73.2 MB/s 
     |████████████████████████████████| 895 kB 65.2 MB/s 
     |████████████████████████████████| 67 kB 7.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!cp -r /content/drive/MyDrive/flask/static /content
!cp -r /content/drive/MyDrive/flask/templates /content

In [ ]:
import speech_recognition as sr
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
from transformers import pipeline

In [ ]:
def get_large_audio_transcription_mp3(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    r = sr.Recognizer()
    # open the audio file using pydub
    sound = AudioSegment.from_mp3(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        # Decibels relative to full scale (dBFS or dB FS) is a unit of measurement for amplitude levels in digital systems
        silence_thresh = sound.dBFS-24,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    print()
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
              pass
                #print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                #print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text
  

In [ ]:
def get_extractive(data):
  from gensim.summarization import summarize
  extractive_summary = summarize(data, ratio=0.3, split=False)
  extractive_summary = extractive_summary.replace('\n', ' ').replace('\r', '')
  return extractive_summary

In [ ]:
summarizert5 = pipeline("summarization", model="henryu-lin/t5-large-samsum-deepspeed")

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [ ]:
def get_abstractive(extractive_summary): 
  return summarizert5(extractive_summary)[0]['summary_text']

In [ ]:
import nltk
from nltk import RegexpParser
from nltk.tree import Tree
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import pandas as pd

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
distilbart_summarizer = pipeline("summarization", model="philschmid/distilbart-cnn-12-6-samsum")


Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
def getChunks(s, maxlength):
    start = 0
    end = 0
    while start + maxlength  < len(s) and end != -1:
        end = s.rfind(" ", start, start + maxlength + 1)
        yield s[start:end]
        start = end +1
    yield s[start:]

def distilbart_summary(transcript):
    chunks = getChunks(transcript,int(len(transcript)/12))
    summary = ""
    for n in chunks:  
        summary_text = distilbart_summarizer(n)[0]['summary_text']
        bullet = "•"+ summary_text+" <br>"
        summary += bullet
    return summary

def is_decision(sent):
    wordsList = nltk.word_tokenize(sent)
    postag_sen = nltk.pos_tag(wordsList)
    chunks = get_chunks(postag_sen)
    for chunk in chunks:
      if type(chunk) is Tree and chunk.label() == "Modal-Auxiliary":
        if "will" in sent:
          return True
      elif type(chunk) is Tree and chunk.label() == "VB-Phrase":
        if "decide" in sent:
          return True
      elif type(chunk) is Tree and chunk.label() == "ING":
        if "going" in sent:
          return True
    return False
''' Helper function to find Verb-Phrase '''
def get_chunks(tagged_sent):
    chunkgram = r"""Modal-Auxiliary: {<NN><MD><VB>}   #Will May Might
                    Modal-Auxiliary: {<NN><MD><RB><VB>} # Will not
                    VB-Phrase: {<NN><VBZ><TO>}  #Decides to
                    VB-Phrase: {<NN><VBD><TO>}  #Decided to
                    ING: {<VBG><TO>}   #Going to
                    """ 
    chunkparser = RegexpParser(chunkgram)
    return chunkparser.parse(tagged_sent)

def get_desicions(summary):
    sentence_list = nltk.sent_tokenize(summary)
    decisions = ""
    for sentence in sentence_list: 
        if is_decision(sentence):
            decisions += sentence
    return decisions

In [ ]:
import spacy
from spacy import displacy
NER = spacy.load("en_core_web_sm")
def has_date(raw_text):
    textNER= NER(raw_text)
    for word in textNER.ents:
        if word.label_ == "DATE":
            return True
    return False
def is_reminder(sent):
    wordsList = nltk.word_tokenize(sent)
    postag_sen = nltk.pos_tag(wordsList)
    chunks = get_chunks(postag_sen)
    for chunk in chunks:
      if has_date(sent):
        if type(chunk) is Tree and chunk.label() == "Modal-Auxiliary":
          return True
        elif type(chunk) is Tree and chunk.label() == "ING":
          if "going" in sent:
            return True
    return False

In [ ]:
def get_reminders(summary):
    sentence_list = nltk.sent_tokenize(summary)
    reminders = ""
    for sentence in sentence_list: 
        if is_reminder(sentence):
            reminders += sentence
    return reminders

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 7.9 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=8a28e7150f995617d491346ecf6528295d78e5b2a20a0540f680eeea5939bfe1
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
from flask import Flask, render_template, request, redirect
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

app = Flask(__name__)
run_with_ngrok(app)   
ngrok.set_auth_token("23iOOnjcsQti6szWflumzeuXjAd_231JDEQ4Z2LdPjm9H6hYZ")

@app.route("/", methods=["GET", "POST"])
def index():
    transcript = ""
    abstractive = ""
    extractive = ""
    distilbart = ""
    decisions = ""
    reminders = ""
    if request.method == "POST":
        print("FORM DATA RECEIVED")

        if "file" not in request.files:
            return redirect(request.url)

        file = request.files["file"]
        print(file)
        if file.filename == "":
            return redirect(request.url)

        if file:
            print('Speech to text')
            transcript = get_large_audio_transcription_mp3(file)
            print('extractive summary')
            extractive = get_extractive(transcript)
            print('abstractive summary')
            abstractive = get_abstractive(extractive)
            print('distilbart summary')
            distilbart = distilbart_summary(transcript)
            print('action items')
            decisions = get_desicions(distilbart)
            reminders = get_reminders(distilbart)
    return render_template('index.html', transcript=transcript, decisions=decisions,reminders=reminders,abstractive=abstractive,extractive=extractive)

    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b0bc-35-233-206-42.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Jan/2022 13:53:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2022 13:53:28] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2022 13:53:28] "GET /static/css/mycss.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2022 13:53:29] "GET /favicon.ico HTTP/1.1" 404 -


FORM DATA RECEIVED
<FileStorage: 'Tesla2021cropped.mp3' ('audio/mpeg')>
Speech to text
extractive summary
abstractive summary
distilbart summary


Your max_length is set to 142, but you input_length is only 6. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


action items


127.0.0.1 - - [15/Jan/2022 13:57:26] "POST / HTTP/1.1" 200 -
